In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep

import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [3]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [4]:
vehicle_agencies = vehicles_gtfs['agency'].drop_duplicates().tolist()

In [5]:
vehicles_gtfs['_merge'].value_counts()

both          192
left_only      26
right_only      0
Name: _merge, dtype: int64

In [6]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

# Merging -> Vehicle & GTFS with Black Cat.

In [7]:
#left merge, Black Cat on the left
Test1 = pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['name'], indicator=True)

In [8]:
len(Test1)

798

In [9]:
# Counting left only and both
Test1['_merge'].value_counts()

both          495
left_only     303
right_only      0
Name: _merge, dtype: int64

In [10]:
#finding the agencies that are in left only.
Left_only = Test1[(Test1._merge.str.contains("left_only", case= False))] 
Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()

In [11]:
# Filter out left only matches
m2 = Test1[~Test1.organization_name.isin(Left_orgs)]

In [12]:
#check that filter worked ok
m2.shape

(495, 45)

In [13]:
m2.sample(3)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,_merge
536,2019,Section 5311(f),CA-2018-114 | 0019000012,2019,Eastern Sierra Transit Authority,BCG0001406,Operating Assistance Sliding Scale,300902,64CO19-00894,155179.00,155179.00,86126.70,69052.30,0,Open,Eastern Sierra Transit Authority,9R02-91062,Rural Reporter,73.00,16.00,8.00,0.00,0.00,0.00,0.00,0.00,97.00,7.31,0.00,NaN,0.00,41.00,0.00,10.00,0.00,46.00,0.00,82.00,0.00,46.00,128.00,Eastern Sierra Transit Authority,99.00,"ok,needed,ok,ok,ok,needed,needed,needed,needed",both
28,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Arcata,BCG0000195,Operating Assistance,300902,64BO17-00364,115350.00,140000.00,115350.00,0.00,0,In Development,City of Arcata,9R02-91018,Rural Reporter,5.00,4.00,1.00,0.00,0.00,0.00,0.00,0.00,10.00,8.50,0.00,NaN,0.00,5.00,0.00,3.00,0.00,2.00,0.00,10.00,0.00,2.00,12.00,City of Arcata,18.00,ok,both
66,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000183,Operating Assistance,300902,64BO17-00402,64841.00,64841.00,64841.00,0.00,0,Open,Imperial County Transportation Commission,90226,Full Reporter,46.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,46.00,4.23,142255.12,NaN,0.00,16.00,0.00,3.00,0.00,27.00,0.00,32.00,0.00,27.00,59.00,Imperial County Transportation Commission,138.00,"needed,needed,needed,needed",both


### Failed Matches, crosswalk manually

In [14]:
#making a data frame with only failed merges out out of original Black Cat
fail = df_5311[df_5311.organization_name.isin(Left_orgs)]

In [15]:
fail.shape

(303, 15)

In [16]:
#crosswalk dictionary for replacing organization name
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
 'City of Dinuba ':  'City of Dinuba',
 'Modoc Transportation Agency': 'Modoc Transportation Agency',
 'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
 'Calaveras County Public Works':  'Calaveras Transit Agency',
 'City of Escalon ':  'City of Escalon, dba: eTrans',
 'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
 'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
 'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
 'County of Tulare': 'Tulare County Area Transit',
 'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
 'Kern Regional Transit':  'Kern Regional Transit',
 'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
 'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
 'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
 'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
 'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
 'Sunline Transit Agency':  'SunLine Transit Agency',
 'Tehama County Transit Agency': 'Tehama County',
 'Trinity County Department of Transportation ':  'Trinity County',
 'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
 'Amador Transit':  'Amador Regional Transit System',
 'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
 'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
 'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
 'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [17]:
#failed merges
fail['organization_name'].replace(crosswalk, inplace= True)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#Merging the failed organizations to vehicles 
Test2 = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'], indicator=True)

In [19]:
Test2['_merge'].value_counts()

both          248
left_only      55
right_only      0
Name: _merge, dtype: int64

In [20]:
#finding the agencies that are in left only...make sure these are the ones we aren't sure have any matches.
Left_only = Test2[(Test2._merge.str.contains("left_only", case= False))]
#find failed agencies 

Left_only_orgs= Left_only['organization_name'].drop_duplicates().tolist()
Left_only_orgs

['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Klamath Trinity Non-Emergency Transportation\u200b',
 'Greyhound Lines, Inc.']

In [21]:
#deleting Klamath off since it has GTFS, we only want agencies that do not appear in both GTFS & NTD 
L_only_2_orgs = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [22]:
#appending failed matches to the first data frame
BC_GTFS_NTD = m2.append(Test2, ignore_index=True)

In [23]:
#checking that rows match up
BC_GTFS_NTD.shape

(798, 45)

In [24]:
#checking that organizations are here
BC_GTFS_NTD['organization_name'].nunique()

87

In [25]:
#drop  merge
BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['_merge'])

### Flag agencies that appear in Black Cat Only

In [26]:
def BC_only(row):
    if row.organization_name in Left_only_orgs:
        return '1'
    else: 
        return '0'  

In [27]:
BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)

In [28]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("greyhound", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
578,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,"Greyhound Lines, Inc.",BCG0000173,Operating Assistance - Route 1 Service from Ar...,300902,64CO17-00447,161293.00,247887.00,161293.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Replacing Klamath
* Klamath has GTFS but no NTD records

In [29]:
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

In [30]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Klamath", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
544,2017,5311(f) Cont,TBD,2017,Klamath Trinity Non-Emergency Transportation​,BCG0000168,Operating Assistance,300902,64CO17-00467,8344.00,8344.00,8344.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436,needed,1


### Getting ok/needed/research out of original GTFS data frame

In [31]:
#look at original GTFS status for all the different varities...
BC_GTFS_NTD['gtfs_schedule_status'].unique()

array(['ok,needed,needed',
       'needed,needed,ok,ok,needed,ok,ok,needed,needed,needed,needed,needed,ok,ok,ok,needed,ok,needed,needed,needed,needed,needed',
       'ok', 'needed,ok,needed,ok', 'needed,ok,ok,ok',
       'need long-term solution,needed', 'ok,needed', 'needed',
       'ok,needed,ok', 'needed,ok', 'need long-term solution', nan,
       'ok,needed,ok,ok,ok,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed', 'needed,needed,needed,needed',
       'needed,ok,needed,needed,needed,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed,needed',
       'needed,needed,needed,needed,research,needed', 'ok,research',
       'ok,needed,needed,ok,needed,needed,research,needed',
       'ok,ok,ok,needed,needed', 'needed,ok,needed',
       'ok,ok,ok,ok,needed,needed',
       'ok,needed,needed,needed,research,needed',
       'ok,ok,ok,ok,ok,ok,needed,needed', 'needed,needed,ok,ok',
       'needed,needed,needed', 'needed,ok,needed,needed', 'ok,ok',
       '

In [32]:
#create a new column
temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))


### Rating Fleet Size
* Using the aggregated df below, I did aggregate1['total_vehicles'].describe() and looked at the different percentiles to choose small, med, large, and no info

In [33]:
#First grabbing only one row for each agency.
Fleet_size = BC_GTFS_NTD.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()

In [34]:
type(Fleet_size)

pandas.core.frame.DataFrame

In [35]:
Fleet_size['total_vehicles'].describe()

count     78.00
mean      89.69
std      189.60
min        1.00
25%        9.25
50%       24.50
75%       80.25
max     1114.00
Name: total_vehicles, dtype: float64

In [36]:
#Get percentiles in a variable
p75 = Fleet_size.total_vehicles.quantile(0.75).astype(float)
p25 = Fleet_size.total_vehicles.quantile(0.25).astype(float)
p50 = Fleet_size.total_vehicles.quantile(0.50).astype(float)


In [37]:
p25

9.25

In [38]:
#Function
def fleet_size (row):
            if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
                return "Small"
            elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
                return "Medium"
            elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
                return "Large"
            else:
                return "No Info"

In [39]:
BC_GTFS_NTD["fleet_size"] = BC_GTFS_NTD.apply(lambda x: fleet_size(x), axis=1)

In [40]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Shafter", case= False))].head(2)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes,GTFS_schedule_status_use,fleet_size
34,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Shafter,BCG0000177,Operating Assistance,300902,64BO17-00383,55562.00,55562.00,55562.00,0.00,0,Open,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,Rural Reporter,7.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,9.00,7.50,0.00,NaN,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,0.00,9.00,9.00,City of Shafter,303.00,needed,0,Needed,Small
116,2017,Section 5311,CA-2017-168 | 0017000259,2017,City of Shafter,BCG0001059,Operating Assistance Sliding Scale,300902,64BO17-00547,56127.00,56127.00,56127.00,0.00,0,Open,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,Rural Reporter,7.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,9.00,7.50,0.00,NaN,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,0.00,9.00,9.00,City of Shafter,303.00,needed,0,Needed,Small


In [41]:
BC_GTFS_NTD.fleet_size.value_counts()

Medium     385
Large      200
Small      158
No Info     55
Name: fleet_size, dtype: int64

### Final Cleaning

In [42]:
#delete old columns
BC_GTFS_NTD2 = BC_GTFS_NTD.drop(columns=['gtfs_schedule_status','name','agency'])

#rename
BC_GTFS_NTD2 = BC_GTFS_NTD2.rename(columns = {'GTFS_schedule_status_use':'GTFS'})

In [43]:
#get agencies without any data to show up 
show_up = ['reporter_type']
for i in show_up:
    BC_GTFS_NTD2[i] = BC_GTFS_NTD2[i].fillna('None')

In [44]:
#change itp id to be float.
BC_GTFS_NTD2['itp_id'] = BC_GTFS_NTD2['itp_id'].fillna(0)
BC_GTFS_NTD2.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436

### Parquet

In [45]:
#Parquet
BC_GTFS_NTD2.to_parquet("BC_GTFS_NTD.parquet")
#Adding to GCS (just in case)
#BC_GTFS_NTD2.to_parquet("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.parquet")
#BC_GTFS_NTD2.to_csv("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.csv")

### Script Testing for FULL merged data frame
* Full = all the years, UPIN, contract, etc

In [46]:
TEST_crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
     'City of Dinuba ':  'City of Dinuba',
     'Modoc Transportation Agency': 'Modoc Transportation Agency',
     'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
     'Calaveras County Public Works':  'Calaveras Transit Agency',
     'City of Escalon ':  'City of Escalon, dba: eTrans',
     'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
     'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
     'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
     'County of Tulare': 'Tulare County Area Transit',
     'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
     'Kern Regional Transit':  'Kern Regional Transit',
     'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
     'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
     'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
     'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
     'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
     'Sunline Transit Agency':  'SunLine Transit Agency',
     'Tehama County Transit Agency': 'Tehama County',
     'Trinity County Department of Transportation ':  'Trinity County',
     'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
     'Amador Transit':  'Amador Regional Transit System',
     'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
     'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
     'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
     'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

#Agencies that ONLY appear in Black Cat
BC_Agency_List = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [47]:
#NOTE TO SELF: once finalized, change TEST-crosswalk back to just crosswalk.
def merged_dataframe():
    ### MERGING ###
    #call the 3 data frames we have to join.
    df_5311 = data_prep.load_grantprojects()
    vehicles = data_prep.load_cleaned_vehiclesdata()
    organizations = data_prep.load_cleaned_organizations_data()
    #merge vehicles from NTD & GTFS
    vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'])
    #left merge, Black Cat on the left and vehicle_gtfs on the right. 
    m1 = (pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], 
                      right_on=['name'], indicator=True)
            )
    
    #Filter out for left only matches, make it into a list
    Left_only = m1[(m1._merge.str.contains("left_only", case= False))] 
    Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()
    #Delete  left only matches from original df 
    m2 = m1[~m1.organization_name.isin(Left_orgs)]
    #making a data frame with only failed merges out out of original Black Cat
    fail = df_5311[df_5311.organization_name.isin(Left_orgs)]
    #replacing organization names from Black Cat with agency names from NTD Vehicles.
    fail['organization_name'].replace(crosswalk, inplace= True)
    #Merging the failed organizations to vehicles 
    Test = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'])
    #appending failed matches to the first data frame
    BC_GTFS_NTD = m2.append(Test, ignore_index=True)
    
    ### METRIC TO SHOW BLACK CAT ONLY AGENCIES ###
    #Function
    def BC_only(row):
        if row.organization_name in BC_Agency_List:
            return '1'
        else: 
            return '0'  
    BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)
    
    #Replace GTFS and cal itp for Klamath. 
    #Klamath does not appear in NTD data so we missed it when we merged NTD & Cal ITP on NTD ID
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"
    
    #get a more definitive GTFS status: ok, needed, long term solution needed, research
    temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
    BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))
    
    ###FLEET SIZE METRIC ### 
    #First grabbing only one row for each agency into a new data frame 
    Fleet_size = BC_GTFS_NTD.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()
    #Get percentiles in objects for total vehicle.
    p75 = Fleet_size.total_vehicles.quantile(0.75).astype(int)
    p25 = Fleet_size.total_vehicles.quantile(0.25).astype(int)
    p50 = Fleet_size.total_vehicles.quantile(0.50).astype(int)
    #Function for fleet size
    def fleet_size (row):
        if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
            return "Small"
        elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
            return "Medium"
        elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
               return "Large"
        else:
            return "No Info"
    BC_GTFS_NTD["fleet_size"] = BC_GTFS_NTD.apply(lambda x: fleet_size(x), axis=1)
    
    ### FINAL CLEAN UP ###
    #delete old columns
    BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['gtfs_schedule_status','name','agency'])
    #rename
    BC_GTFS_NTD = BC_GTFS_NTD.rename(columns = {'GTFS_schedule_status_use':'GTFS'})
    #get agencies without any data to show up when grouping
    show_up = ['reporter_type']
    for i in show_up:
        BC_GTFS_NTD[i] = BC_GTFS_NTD[i].fillna('None')
    #change itp id to be floats & 0 so parquet will work
    BC_GTFS_NTD['itp_id'] = BC_GTFS_NTD['itp_id'].fillna(0)
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436
    #save into parquet
    BC_GTFS_NTD2.to_parquet("BC_GTFS_NTD.parquet")
    return BC_GTFS_NTD 

In [48]:
test = merged_dataframe()

test.sample(5)

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,_merge,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size
513,2016,Section 5311,CA-2017-025 | 0017000158,2016,Glenn County Transportation Commission,BCG0000230,Operating Assistance,300902,64BO17-00400,150218.00,150218.00,150218.00,0.00,0,Open,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,None,No Info
207,2018,Section 5311,CA-2018-114 | 0019000012,2018,City of Tehachapi,BCG0001040,Operating Assistance Sliding Scale,300902,64BO19-00870,38861.00,38861.00,38861.00,0.00,0,Open,9R02-91078,Rural Reporter,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,5.67,0.00,NaN,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,4.00,4.00,333.00,both,0,None,Small
631,2018,Section 5311,CA-2018-114 | 0019000012,2018,"County of Sonoma, dba: Sonoma County Transit",BCG0001276,Purchase Replacement < 30 Ft Bus,111204,64BC19-00837,67357.00,67357.00,67357.00,0.00,0,Open,90089,Full Reporter,65.00,22.00,6.00,1.00,0.00,0.00,0.00,0.00,94.00,5.92,173050.20,NaN,5.00,46.00,0.00,12.00,0.00,31.00,10.00,92.00,0.00,31.00,133.00,314.00,NaN,0,Ok,Large
688,2019,Section 5311,CA-2020-015 | 0020000057,2019,Tehama County,BCG0001497,Purchase Replacement < 30 Ft Bus,111204,64BC19-00970,360108.00,713772.00,0.00,360108.00,0,Open,9R02-91089,Rural Reporter,7.00,2.00,0.00,3.00,0.00,0.00,0.00,0.00,12.00,7.40,0.00,NaN,0.00,2.00,0.00,2.00,0.00,8.00,0.00,4.00,0.00,8.00,12.00,0,NaN,0,None,Medium
171,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,Monterey-Salinas Transit,BCG0000277,BUY 40-FT BUS FOR EXPANSION,111301,64CC17-00452,265590.00,288473.00,265590.00,0.00,0,Open,90062,Full Reporter,151.00,15.00,14.00,16.00,0.00,0.00,0.00,0.00,196.00,7.13,317142.10,NaN,0.00,93.00,0.00,38.00,0.00,65.00,0.00,186.00,0.00,65.00,251.00,208.00,both,0,Research,Large


### Testing 2/28
* Duplicating allocation amount col so we can get the mean & total across the years for each organization.

In [56]:
def aggregated_merged_df(): 
    #read original df & airtable
    df_original = data_prep.merged_dataframe()
    airtable = data_prep.load_airtable()
    
    # Grab the reporter type, fleet size, ntd id, and itp id to subset out
    df_subset = df_original[['organization_name','reporter_type', 'fleet_size', 'ntd_id', 'itp_id']]
    
    #aggregate vehicles 16+ above. 
    column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
    df_original['_16plus']= df_original[column_names].sum(axis=1)
    
    #duplicate allocationamount col so we can get the mean an organiztaion received.
    df_original['allocationamount_mean'] = df_original['allocationamount']
    
    #grab the max of all vehicle information & sum of all monetary cols from Black Cat
    df_original = df_original.groupby(['organization_name',]).agg({'allocationamount_mean': 
                                                                   'mean',
                                                                   'allocationamount':'sum',
                                         'encumbered_amount':'sum',
                                         'expendedamount':'sum',
                                         'activebalance':'sum',
                                         'closedoutbalance':'sum',
                                         'adjusted_allocationamount':'sum',
                                         'adjusted_expendedamount':'sum',
                                         'adjusted_encumbered_amount':'sum',
                                         'adjusted_activebalance':'sum',
                                         'adjusted_closedoutbalance':'sum',                                                
                                         'total_vehicles':'max',
                                         'average_age_of_fleet__in_years_':'max',
                                         'average_lifetime_miles_per_vehicle':'max',
                                         'Automobiles':'max',
                                         'Bus':'max',
                                         'Other':'max',
                                         'Train':'max',
                                         'Van':'max',
                                         'automobiles_door':'max',
                                         'bus_doors':'max',
                                         'van_doors':'max',
                                         'train_doors':'max',
                                         'doors_sum':'max',
                                         '_0_9':'max',
                                         '_10_12':'max',
                                         '_13_15':'max',
                                         '_16_20':'max',
                                         '_21_25':'max',
                                         '_26_30':'max',
                                         '_31_60':'max',
                                         '_16plus':'max',
                                         '_60plus':'max'}).reset_index()
   
   
    df_subset = df_subset.drop_duplicates()
    
    #Merge the 2 dfs together
    df_merge1 = pd.merge(df_original,df_subset, how='left', left_on='organization_name', right_on = 'organization_name')
    
    #Merge df_merge1 with Air table that contains MPO, planning authority, more definitive GTFS status
    df_merge2 = pd.merge(df_merge1, airtable, how='left', left_on='organization_name', right_on='name')
    
    #GTFS is now split between gtfs_static & gtfs_realtime, create groups to show orgs with both
    #orgs with only either RT/Static, orgs with none
    def gtfs_status_indicator(df):   
        if (df['gtfs_static_status'] == 'Static OK') and (df['gtfs_realtime_status'] == "RT OK"):
            return 'Both static & RT OK'
        elif (df['gtfs_static_status'] == 'Static Incomplete') and (df['gtfs_realtime_status'] == "RT OK"):
            return 'Only RT'
        elif (df['gtfs_static_status'] == 'Static OK') and (df['gtfs_realtime_status'] == "RT Incomplete"):
            return 'Only Static'
        else: 
            return "None"
    df_merge2['GTFS_Status'] = df_merge2.apply(gtfs_status_indicator, axis = 1)
    
    #Drop unnecessary cols
    df_merge2 = df_merge2.drop('name', 1)
    
    return df_merge2 

In [57]:
test = aggregated_merged_df()

/tmp/ipykernel_2717/1939546232.py:75: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [58]:
test

,organization_name,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,reporter_type,fleet_size,ntd_id,itp_id,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,GTFS_Status
0,Alpine County Community Development,341366.00,293366.00,159901.31,181464.69,0,363613.49,173611.38,315613.45,190002.10,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,None,No Info,NaN,0,NaN,NaN,NaN,NaN,NaN,None
1,Amador Regional Transit System,1518539.00,1279114.00,1067984.00,450555.00,0,1612119.29,1151645.28,1372694.08,460474.01,0.00,24.00,4.07,0.00,0.00,0.00,0.00,0.00,21.00,0.00,0.00,21.00,0.00,21.00,18.00,3.00,1.00,1.00,1.00,0.00,0.00,2.00,0.00,Rural Reporter,Medium,9R02-91000,11.00,10 - Stockton,NaN,Amador County Transportation Commission,Static OK,RT Incomplete,Only Static
2,Butte County Association of Governments,5886587.00,4820850.00,4362924.78,1523662.22,0,6267702.55,4719990.91,5201964.65,1547711.65,0.00,53.00,6.23,200297.84,0.00,29.00,0.00,0.00,24.00,0.00,58.00,24.00,0.00,82.00,44.00,9.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Full Reporter,Medium,90208,47.00,03 - Marysville,Butte County Association of Governments,Butte County Association of Governments,Static OK,RT Incomplete,Only Static
3,Calaveras Transit Agency,1688814.00,1914947.00,1081593.98,607220.02,0,1801548.92,1161866.84,2058384.96,639682.08,0.00,12.00,5.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,0.00,11.00,0.00,11.00,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Rural Reporter,Medium,9R02-99442,0,NaN,NaN,NaN,NaN,NaN,None
4,"Central Contra Costa Transit Authority, dba: C...",50412.00,50412.00,0.00,50412.00,0,54399.71,0.00,54399.71,54399.71,0.00,206.00,3.44,72814.41,0.00,125.00,0.00,0.00,62.00,0.00,250.00,62.00,0.00,312.00,160.00,43.00,1.00,1.00,1.00,0.00,0.00,2.00,0.00,Full Reporter,Large,90078,0,NaN,NaN,NaN,NaN,NaN,None
5,City of Arcata,574780.00,1004780.00,574780.00,0.00,0,628747.61,628747.61,1110688.90,0.00,0.00,10.00,8.50,0.00,0.00,5.00,0.00,0.00,2.00,0.00,10.00,2.00,0.00,12.00,5.00,4.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Rural Reporter,Medium,9R02-91018,18.00,01 - Eureka,NaN,Humboldt County Association of Governments,Static OK,RT OK,Both static & RT OK
6,City of Arvin,412717.00,336683.00,263453.00,149264.00,0,440489.23,287784.85,364455.16,152704.38,0.00,10.00,6.58,0.00,0.00,6.00,0.00,0.00,3.00,0.00,12.00,3.00,0.00,15.00,9.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Rural Reporter,Medium,9R02-91027,21.00,06 - Fresno,Kern Council of Governments,Kern Council of Governments,Static OK,RT Incomplete,Only Static
7,City of Auburn,419195.00,358817.00,300665.00,118530.00,0,449850.39,328588.38,389472.34,121262.01,0.00,6.00,6.25,0.00,0.00,2.00,0.00,0.00,3.00,0.00,4.00,3.00,0.00,7.00,5.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,Rural Reporter,Small,9R02-91032,23.00,03 - Marysville,Sacramento Area Council of Governments,Placer County Transportation Planning Agency,Static OK,RT Incomplete,Only Static
8,City of California City,276006.00,224769.00,175422.00,100584.00,0,294482.04,191579.69,243245.00,102902.35,0.00,4.00,11.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,8.00,0.00,0.00,8.00,0.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Rural Reporter,Small,9R02-91111,51.00,09 - Bishop,Kern Council of Governments,Kern Council of Governments,Static OK,RT Incomplete,Only Static
9,"City of Chowchilla, dba: Chowchilla Area Transit",444381.00,485945.00,371434.00,72947.00,0,477612.76,404665.70,533949.54,72947.06,0.00,4.00,5.75,0.00,0.00,0.00,0.00,0.00,4.00,0.00,0.00,4.00,0.00,4.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Rural Reporter,Small,9R02-91071,65.00,NaN